In [1]:
import requests
import json
import xml.etree.ElementTree as ET

def get_osm_map_bounds(osm_map_dir):
    bounds = ET.parse(osm_map_dir).getroot().find("bounds")
    return dict(bounds.attrib.items())

def get_region_issues(osm_map_dir, city_key, labels, out_dir):
    coords = get_osm_map_bounds(osm_map_dir)
    resp = requests.get(
        f"https://sidewalk-{city_key}.cs.washington.edu/v3/api/labelClusters",
        params={ "bbox": f"{coords["minlon"]},{coords["minlat"]},{coords["maxlon"]},{coords["maxlat"]}", "labelType": ",".join(labels) },
        timeout=20
    )
    if resp.status_code == 200:
        data = resp.json()
        with open(out_dir, "w") as f:
            json.dump(data, f, indent=4)

get_region_issues("./test_data/small_map.osm", "sea", ["NoCurbRamp", "SurfaceProblem", "NoSidewalk"], "labels.json")

In [ ]:
import datetime
import json
import numpy as np
import pandas as pd

EARTH_RAD = 6_378_137

def get_region_info(json_path, num_days=365*5):
    with open(json_path, "r") as f:
        file = json.load(f)
    markers = []
    if "features" in file.keys():
        for f in file["features"]:
            date = f["properties"]["avg_image_capture_date"].split("-")
            if (datetime.datetime.now() - datetime.datetime(int(date[0]), int(date[1]), int(date[2][:2]))).days <= num_days:
                markers.append((f["properties"]["label_type"], f["geometry"]["coordinates"])) 
    return markers

def get_translation_from_center(center_coords, cur_coords):
    center_lng, center_lat = center_coords
    cur_lng, cur_lat = cur_coords
    lat_diff = np.deg2rad(cur_lat - center_lat)
    lng_diff = np.deg2rad(cur_lng - center_lng)
    # local tangent plane approximation
    x = lng_diff * EARTH_RAD * np.cos(np.deg2rad(center_lat))
    y = lat_diff * EARTH_RAD
    return (x, y)

def centered_region_info_to_csv(markers, osm_map_dir, out_path):
    bounds = get_osm_map_bounds(osm_map_dir)
    center_lng, center_lat = (float(bounds["minlon"]) + float(bounds["maxlon"])) / 2, (float(bounds["minlat"]) + float(bounds["maxlat"])) / 2

    df = pd.DataFrame(columns=["issue label", "dx", "dy", "lng", "lat"])
    for m in markers:
        lng, lat = m[1]
        dx, dy = get_translation_from_center((center_lng, center_lat), (lng, lat))
        df.loc[len(df)] = [m[0], dx, dy, lng, lat]
    df.to_csv(out_path)


centered_region_info_to_csv(get_region_info("labels.json", 365*20), "test_data/small_map.osm", "translations.csv")

In [ ]:
import osmnx as ox
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
import math

EARTH_RAD = 6_378_137
walkable_edges = ["footway", "path", "pedestrian", "residential", "living_street", "service"]

def cost(coord1, coord2, lat_center):
    dx = (coord2['x'] - coord1['x']) * math.pi/180 * EARTH_RAD * math.cos(math.radians(lat_center))
    dy = (coord2['y'] - coord1['y']) * math.pi/180 * EARTH_RAD
    return math.sqrt(dx ** 2 + dy ** 2)


# parses OSM file to extract nodes for A* implementation and define adjacency map, filtered to walkable edges
def parse_osm_file(path, marker_csv_path):
    G = ox.graph_from_xml(filepath=path, simplify=True)
    print("num nodes: {}, num edges: {}".format(len(G.nodes()), len(G.edges())))

    nonwalkable_edges = []
    for u, v, k, data in G.edges(keys=True, data=True):
        highway = data.get("highway")
        if highway is None or highway not in walkable_edges:
            nonwalkable_edges.append((u, v, k))
    G.remove_edges_from(nonwalkable_edges)

    # add marker edges
    df = pd.read_csv(marker_csv_path)
    all_lats = [G.nodes[n]['y'] for n in G.nodes]
    lat_center = np.mean(all_lats)

    for i in range(len(df)):
        id = max(G.nodes) + 1
        issue_x, issue_y = df.loc[i, "lng"], df.loc[i, "lat"]

        # find and connect the two nearest nodes to the marker
        coords = np.array([[G.nodes[n]['x'], G.nodes[n]['y']] for n in G.nodes])
        tree = cKDTree(coords)
        _, indices = tree.query([issue_x, issue_y], 2)
        G.add_node(id, x=issue_x, y=issue_y, issue=True)

        nearest_nodes = [list(G.nodes)[index] for index in indices]
        for node in nearest_nodes:
            c = cost(G.nodes[id], G.nodes[node], lat_center)
            G.add_edge(id, node, length=c)
            G.add_edge(node, id, length=c)

    node_locs_from_center = {}
    for node_id, data in G.nodes(data=True):
        x = data.get('x')
        y = data.get('y')
        bounds = get_osm_map_bounds(path)
        center_lng, center_lat = (float(bounds["minlon"]) + float(bounds["maxlon"])) / 2, (float(bounds["minlat"]) + float(bounds["maxlat"])) / 2
        dx, dy = get_translation_from_center((center_lng, center_lat), (x, y))
        print(f"Node {node_id}: x = {dx}, y = {dy}")
        node_locs_from_center[node_id] = (dx, dy)

    adj_map = {}
    for u, v, data in G.edges(data=True):
        length = data.get('length')
        if u not in adj_map:
            adj_map[u] = []
        adj_map[u].append((v, length))

    return G, adj_map, node_locs_from_center
    
G, adj_map, node_locs = parse_osm_file("/Users/irislitiu/Downloads/small_map.osm", "./translations.csv")

for node_id, (dx, dy) in node_locs.items():
    if G.nodes[node_id].get("issue", False):
        print(f"Marker node {node_id} at dx={round(dx, 2)}, dy={round(dy, 2)}")
    else:
        print(f"Regular node {node_id} at dx={round(dx, 2)}, dy={round(dy, 2)}")


num nodes: 312, num edges: 948
Node 12234207233: x = -276.0246384771844, y = 54.10127252577663
Node 12289740803: x = -365.6747552235882, y = -52.89902202460619
Node 12234207236: x = -160.16818065836003, y = 95.80155377701136
Node 12289740806: x = -276.27943275784213, y = 28.386470152594317
Node 12289740810: x = -181.80321259390294, y = 95.05571318868117
Node 12289740811: x = -363.2841853633018, y = 28.66476887887486
Node 12289740813: x = -365.84711606034716, y = 12.389859325332473
Node 12234207240: x = -160.3929991399778, y = 111.4196783351079
Node 12234207255: x = -181.63085175714397, y = 257.8493365241066
Node 12234207254: x = -159.87591663076597, y = 258.12763525117816
Node 12289740826: x = -303.1227595303096, y = 13.157963812081507
Node 12289740827: x = -303.6548299385089, y = 28.34194235575665
Node 12291287068: x = 426.36075751581836, y = -146.30720674927707
Node 12291287074: x = 242.16697506056408, y = -99.21906214396259
Node 12291287075: x = 242.4292632901831, y = -133.204902683

In [ ]:
import json

def adjacency_map_to_json(G, adj_map, node_locs, out_path):
    nodes_json = {}
    for node_id, (dx, dy) in node_locs.items():
        nodes_json[node_id] = {
            "x": dx,
            "y": dy,
            "issue": G.nodes[node_id].get("issue", False)
        }

    adj_json = {}
    for u, neighbors in adj_map.items():
        adj_json[u] = [{"to": v, "cost": length} for v, length in neighbors]
    graph_json = {
        "nodes": nodes_json,
        "adjacency": adj_json
    }

    with open(out_path, "w") as f:
        json.dump(graph_json, f, indent=2)